In [6]:
from matplotlib import pyplot as plt

import pandas as pd
import numpy as np
import seaborn as sns

import csv
import copy
import datetime
import math

In [7]:
def str_to_datetime(timestamp):
  date_parts = timestamp.split('-')
  year, month, day = int(date_parts[0]), int(date_parts[1]), int(date_parts[2])
  return datetime.datetime(year=year, month=month, day=day)

In [8]:
def write_rows_on_csv(path, data):
    with open(path, "a", newline="") as file:
        writer = csv.writer(file)

        # Write new rows to the CSV file
        new_rows = data
        writer.writerows(new_rows)

In [13]:
def build_df(ticker):
    """
        Build data-frame attaching indexes to historical values

        Parameters:
        ticker (string): stock ticker to build df

        Returns:
        Pandas Data-frame: df
    """
    
    df = pd.read_csv(f'data/stock-data/{ticker}.SAO.csv')

    df["timestamp"] = df["timestamp"].apply(str_to_datetime)
    df.index = df.pop("timestamp")

    df_reversed = df[::-1]
    df_reversed['7_m_average'] = df_reversed['close'].rolling(window=7).mean()
    df_reversed['40_m_average'] = df_reversed['close'].rolling(window=40).mean()
    df_result = df_reversed[::-1] 
    df_result = df_result.dropna(subset=['7_m_average', '40_m_average'])

    return df_result[::-1]

In [20]:
def crossing_averages(row, idx, previous_trend, output_path, ic):
    """
        Checks want move to make given crossing average strategy

        Parameters:
        row (data-frame row): row from specific day of a data-frame
        idx (data-frame row index): index from specific row of a data-frame
        previous_trend (int): 1 for up trend and -1 for down trend for the previous day
        output_path (string): path of the results sheet
        ic (float): initial capital to simulate

        Returns:
        int: 1 for up trend and -1 for down trend for the current day
    """

    if(row['40_m_average'] >= row['7_m_average']): #sell
    
        if(previous_trend != -1): #trend_change

            output_df = pd.read_csv(output_path)

            if(len(output_df) >= 1): #not empty sheet

                respective_purchase = output_df.iloc[-1]

                price = row['close']

                rescued = respective_purchase['n_stock'] * row['close']

                write_rows_on_csv(
                    output_path, 
                    [[idx, 'sell', row['close'], rescued, respective_purchase['n_stock'], respective_purchase['remainder']]]
                    )

                skip_days = 2
            
                return -1

            else: return previous_trend

        else: return previous_trend
            
    else: #buy

        if(previous_trend != 1): #trend_change

            output_df = pd.read_csv(output_path)

            last_sell = output_df.iloc[-1] if(len(output_df) > 1) else 0

            capital = ic if(len(output_df) <= 1) else last_sell['invested/rescued'] + last_sell['remainder']

            n_stock = capital // row['close']

            remainder = capital % row['close']

            write_rows_on_csv(
                output_path, 
                [[idx, 'buy', row['close'], capital - remainder, n_stock, remainder]]
                )

            return 1

        else: return previous_trend

In [11]:
def simulate(ticker, ic):

    df = build_df(ticker)
   
    previous_trend = 0

    skip_days = 0

    output_path = (f'results/{ticker}_m_av_sim.csv')

    with open(output_path, mode="w", newline="") as file:
        writer = csv.writer(file)

        # write header
        writer.writerow(
            ['date', 'move', 'price', 'invested/rescued', 'n_stock', 'remainder']
        )

    for idx, row in df.iterrows():

        if (skip_days > 0):
            skip_days -= 1
            continue

        previous_trend = crossing_averages(row, idx, previous_trend, output_path, ic)

In [21]:
simulate('BBAS3', 1000)

/tmp/ipykernel_7785/1383793422.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reversed['7_m_average'] = df_reversed['close'].rolling(window=7).mean()
/tmp/ipykernel_7785/1383793422.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reversed['40_m_average'] = df_reversed['close'].rolling(window=40).mean()


In [ ]:
11 % 3